In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
#from source.utils import *
from source.preprocess import *
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

In [3]:
path = "../../Desktop/DLC_social_1/"

# Set up and design the project

In [4]:
with open('{}DLC_social_1_exp_conditions.pickle'.format(path), 'rb') as handle:
    Treatment_dict = pickle.load(handle)

In [5]:
#Which angles to compute?
bp_dict = {'B_Nose':['B_Left_ear','B_Right_ear'],
          'B_Left_ear':['B_Nose','B_Right_ear','B_Center','B_Left_flank'],
          'B_Right_ear':['B_Nose','B_Left_ear','B_Center','B_Right_flank'],
          'B_Center':['B_Left_ear','B_Right_ear','B_Left_flank','B_Right_flank','B_Tail_base'],
          'B_Left_flank':['B_Left_ear','B_Center','B_Tail_base'],
          'B_Right_flank':['B_Right_ear','B_Center','B_Tail_base'],
          'B_Tail_base':['B_Center','B_Left_flank','B_Right_flank']}

In [6]:
%%time
DLC_social_1 = project(path=path,#Path where to find the required files
                   smooth_alpha=0.85,                    #Alpha value for exponentially weighted smoothing
                   distances=['B_Center','B_Nose','B_Left_ear','B_Right_ear','B_Left_flank',
                              'B_Right_flank','B_Tail_base'],
                   ego=False,
                   angles=True,
                   connectivity=bp_dict,
                   arena='circular',                  #Type of arena used in the experiments
                   arena_dims=[380],                  #Dimensions of the arena. Just one if it's circular
                   video_format='.mp4',
                   table_format='.h5',
                   exp_conditions=Treatment_dict)

CPU times: user 2.6 s, sys: 837 ms, total: 3.43 s
Wall time: 1.19 s


# Run project

In [7]:
%%time
DLC_social_1_coords = DLC_social_1.run(verbose=True)
print(DLC_social_1_coords)
type(DLC_social_1_coords)

Loading trajectories...
Smoothing trajectories...
Computing distances...
Computing angles...
Done!
Coordinates of 47 videos across 4 conditions
CPU times: user 10.6 s, sys: 711 ms, total: 11.3 s
Wall time: 11.8 s


source.preprocess.coordinates

# Generate coords

In [8]:
%%time
ptest = DLC_social_1_coords.get_coords(center=True, polar=False, speed=0, length='00:10:00')
ptest._type

CPU times: user 891 ms, sys: 70.4 ms, total: 962 ms
Wall time: 896 ms


'coords'

In [9]:
%%time
dtest = DLC_social_1_coords.get_distances(speed=0, length='00:10:00')
dtest._type

CPU times: user 564 ms, sys: 411 ms, total: 975 ms
Wall time: 988 ms


'dists'

In [10]:
%%time
atest = DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00')
atest._type

CPU times: user 140 ms, sys: 84.8 ms, total: 224 ms
Wall time: 226 ms


'angles'

# Visualization playground

In [11]:
#ptest.plot_heatmaps(['B_Center', 'W_Center'], i=1)

In [12]:
#Plot animation of trajectory over time with different smoothings
#plt.plot(ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['x'],
#         ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['y'], label='alpha=0.85')

#plt.xlabel('x')
#plt.ylabel('y')
#plt.title('Mouse Center Trajectory using different exponential smoothings')
#plt.legend()
#plt.show()

# Dimensionality reduction playground

In [13]:
#pca = ptest.pca(4, 1000)

In [14]:
#plt.scatter(*pca[0].T)
#plt.show()

# Preprocessing playground

In [15]:
mtest = merge_tables(DLC_social_1_coords.get_coords(center=True, polar=True, length='00:10:00'))#,
#                      DLC_social_1_coords.get_distances(speed=0, length='00:10:00'),
#                      DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00'))

In [16]:
#pptest = mtest.preprocess(window_size=51, filter='gaussian', sigma=10, shift=20)

In [32]:
pttest = mtest.preprocess(window_size=51, window_step=10, filter=None)
pttest.shape

(70500, 51, 28)

In [18]:
#plt.plot(pttest[2,:,2], label='normal')
#plt.plot(pptest[2,:,2], label='gaussian')
#plt.legend()
#plt.show()

# Trained models playground

### Seq 2 seq Variational Auto Encoder

In [19]:
from datetime import datetime
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.constraints import UnitNorm
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
from tensorflow.keras.layers import Lambda, LSTM
from tensorflow.keras.layers import RepeatVector, TimeDistributed
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from source.model_utils import *
import keras as k
import tensorflow as tf

Using TensorFlow backend.


In [20]:
NAME = 'Baseline_VAE_long'
log_dir = os.path.abspath(
    "logs/fit/{}_{}".format(NAME, datetime.now().strftime("%Y%m%d-%H%M%S"))
)
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
from source.models import SEQ_2_SEQ_AE, SEQ_2_SEQ_VAE

In [22]:
#encoder, decoder, ae = SEQ_2_SEQ_AE(pttest.shape).build()

In [23]:
encoder, generator, vae = SEQ_2_SEQ_VAE(pttest.shape).build()

In [24]:
#ae.summary()
#vae.summary()

In [25]:
#from tensorflow.keras.utils import plot_model
#plot_model(encoder)

In [26]:
#Shuffle to check if ordered sequence makes it harder 
#for the model in the validation set!
#np.random.shuffle(pttest)
pttrain = pttest[:-1500]
pttest  = pttest[-1500:]

In [27]:
#tf.config.experimental_run_functions_eagerly(False)
history = vae.fit(pttrain, pttrain, epochs=100, batch_size=256, verbose=1, validation_data=(pttest, pttest),
                 callbacks=[tensorboard_callback])